# 用元类来验证子类

In [1]:
import logging
from pprint import pprint

元类最简单的一种用途，就是验证某个类定义是否正确。

In [2]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        orig_print = __builtins__.print
        print = pprint
        print((meta, name, bases, class_dict))
        print = orig_print
        return type.__new__(meta, name, bases, class_dict)

In [3]:
class MyClass(object, metaclass=Meta):
    stuff = 123

    def foo(self):
        pass

(<class '__main__.Meta'>,
 'MyClass',
 (<class 'object'>,),
 {'__module__': '__main__',
  '__qualname__': 'MyClass',
  'foo': <function MyClass.foo at 0x00000000153952F0>,
  'stuff': 123})


元类可以获知那个类的名称、其所继承的父类，以及定义在class语句体中的全部类属性。

**示例：**用类表示任意多边形。验证多边形至少有3条边。

In [4]:
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        # Don't validate the abstract Polygon class
        if bases != (object,):
            if class_dict['sides'] < 3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)

In [5]:
class Polygon(object, metaclass=ValidatePolygon):
    sides = None  # Specified by subclasses

    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180

In [6]:
class Triangle(Polygon):
    sides = 3

In [7]:
print(Triangle.interior_angles())

180


假如尝试定义一种边数小于3的多边形子类，那么class语句体刚一结束，元类中的验证代码立刻就会拒绝这个class。

In [8]:
try:
    print('Before class')
    class Line(Polygon):
        print('Before sides')
        sides = 1
        print('After sides')
    print('After class')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-8-a78223c6f041>", line 3, in <module>
    class Line(Polygon):
  File "<ipython-input-4-7b5b9141a878>", line 6, in __new__
    raise ValueError('Polygons need 3+ sides')
ValueError: Polygons need 3+ sides


Before class
Before sides
After sides
